In [1]:
from __future__ import print_function, division
import torch
import matplotlib.pyplot as plt
import argparse,os
import pandas as pd
import cv2
import numpy as np
import random
import math
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from models.CDCNs import Conv2d_cd, CDCN, CDCNpp
from Load_OULUNPU_train import Spoofing_train, Normaliztion, ToTensor, RandomHorizontalFlip, Cutout, RandomErasing
from Load_OULUNPU_valtest import Spoofing_valtest, Normaliztion_valtest, ToTensor_valtest
from Load_Single_Img import getTestImg
import torch.nn.functional as F
from torch.autograd import Variable
import torch.nn as nn
import torch.optim as optim
import copy
import pdb
from utils import AvgrageMeter, accuracy, performances
import torchvision.utils as vutils

# Dataset root
train_image_dir = '/mnt/hdd.user/datasets/Oulu-NPU/Train_files/'
val_image_dir = '/mnt/hdd.user/datasets/Oulu-NPU/Dev_files/'
test_image_dir = '/mnt/hdd.user/datasets/Oulu-NPU/Test_files/'
# 文件夹内包含原来的视频.avi文件和其中每一帧的图像1_1_01_1_frame92.jpg
# 以及每一帧的ROi region of interest在.txt文件中 文件内容类似0,455,779,638,756
train_dat_dir = '/mnt/hdd.user/datasets/Oulu-NPU/Train_dat/'
val_dat_dir = '/mnt/hdd.user/datasets/Oulu-NPU/Dev_dat/'
test_dat_dir = '/mnt/hdd.user/datasets/Oulu-NPU/Test_dat/'

map_dir = '/mnt/hdd.user/datasets/Oulu-NPU/Train_depth/'
val_map_dir = '/mnt/hdd.user/datasets/Oulu-NPU/Dev_depth/'
test_map_dir = '/mnt/hdd.user/datasets/Oulu-NPU/Test_depth/'
# 文件名1_2_11_3_frame140_depth.jpg

train_list = '/mnt/hdd.user/datasets/Oulu-NPU/Protocols/Protocol_1/Train.txt'
val_list = '/mnt/hdd.user/datasets/Oulu-NPU/Protocols/Protocol_1/Dev.txt'
test_list = '/mnt/hdd.user/datasets/Oulu-NPU/Protocols/Protocol_1/Test.txt'
# 文件都会在’train.txt’、'Dev.txt’和’Test.txt’中存放训练、验证、测试建议方法的视频文件列表。这些文件的组织如下：

device = torch.device("cuda:8")

In [2]:
class Contrast_depth_loss(nn.Module):    # Pearson range [-1, 1] so if < 0, abs|loss| ; if >0, 1- loss
    def __init__(self):
        super(Contrast_depth_loss,self).__init__()
        return
    def forward(self, out, label): 
        '''
        compute contrast depth in both of (out, label),
        then get the loss of them
        tf.atrous_convd match tf-versions: 1.4
        '''
        # 根据out和label分别构造depth_conv，之后计算mseloss
        contrast_out = contrast_depth_conv(out)
        contrast_label = contrast_depth_conv(label)
        
        
        criterion_MSE = nn.MSELoss().to(device)
    
        loss = criterion_MSE(contrast_out, contrast_label)
        #loss = torch.pow(contrast_out - contrast_label, 2)
        #loss = torch.mean(loss)
    
        return loss
    
criterion_absolute_loss = nn.MSELoss().to(device)
criterion_contrastive_loss = Contrast_depth_loss().to(device)

In [3]:
parser = argparse.ArgumentParser()
parser.add_argument('--gpu', type=int, default=1, help='the gpu id used for predict')
parser.add_argument('--lr', type=float, default=0.0001, help='initial learning rate')  
parser.add_argument('--batchsize', type=int, default=7, help='initial batchsize')  
parser.add_argument('--step_size', type=int, default=500, help='how many epochs lr decays once')  # 500 
parser.add_argument('--gamma', type=float, default=0.5, help='gamma of optim.lr_scheduler.StepLR, decay of lr')
parser.add_argument('--echo_batches', type=int, default=50, help='how many batches display once')  # 50
parser.add_argument('--epochs', type=int, default=1300, help='total training epochs')
parser.add_argument('--log', type=str, default="CDCNpp_P1", help='log and save model name')
parser.add_argument('--finetune', action='store_true', default=False, help='whether finetune other models')

args = parser.parse_args(args=[])
print(args)

epsilon = 1e-2

Namespace(batchsize=7, echo_batches=50, epochs=1300, finetune=False, gamma=0.5, gpu=1, log='CDCNpp_P1', lr=0.0001, step_size=500)


In [4]:
log_file = open(args.log+'/'+ args.log+'_log_val_test.txt', 'w')

log_file.write('Validation and Test:\n')
log_file.flush()

for i in range(11):
    epoch = i * 50 + 1
    model_state_dict = torch.load(f"./CDCNpp_P1/CDCNpp_P1_{epoch}.pkl", map_location={'cuda:7':'cuda:8'})
    model = CDCNpp(basic_conv=Conv2d_cd, theta=0.7)
    model.load_state_dict(model_state_dict)
    model.eval()
    model.to(device)

    with torch.no_grad():
        ###########################################
        '''                val             '''
        ###########################################
        # val for threshold
        val_data = Spoofing_valtest(val_list, val_image_dir, val_map_dir, val_dat_dir, transform=transforms.Compose([Normaliztion_valtest(), ToTensor_valtest()]))
        dataloader_val = DataLoader(val_data, batch_size=1, shuffle=False, num_workers=1)

        map_score_list = []
        criterion_MSE = nn.MSELoss().to(device)

        for i, sample_batched in enumerate(dataloader_val):
            # get the inputs
            inputs, spoof_label = sample_batched['image_x'].to(device), sample_batched['spoofing_label'].to(device)
            val_maps = sample_batched['val_map_x'].to(device)   # binary map from PRNet

            #pdb.set_trace()
            map_score = 0.0
            for frame_t in range(inputs.shape[1]):
                map_x, embedding, x_Block1, x_Block2, x_Block3, x_input =  model(inputs[:,frame_t,:,:,:])
                score_norm = criterion_MSE(map_x, val_maps[:,frame_t,:,:])
                map_score += score_norm
            map_score = map_score/inputs.shape[1]

            map_score_list.append('{} {}\n'.format(map_score, spoof_label[0][0]))

            #pdb.set_trace()
        map_score_val_filename2 = args.log+'/'+ args.log+'_map_score_val2.txt'
        with open(map_score_val_filename2, 'w') as file:
            file.writelines(map_score_list)                

        ###########################################
        '''                test             '''
        ##########################################
        # test for ACC
        test_data = Spoofing_valtest(test_list, test_image_dir, test_map_dir, test_dat_dir, transform=transforms.Compose([Normaliztion_valtest(), ToTensor_valtest()]))
        dataloader_test = DataLoader(test_data, batch_size=1, shuffle=False, num_workers=1)

        map_score_list = []

        for i, sample_batched in enumerate(dataloader_test):
            # get the inputs
            inputs, spoof_label = sample_batched['image_x'].to(device), sample_batched['spoofing_label'].to(device)
            test_maps = sample_batched['val_map_x'].to(device)   # binary map from PRNet 


            #pdb.set_trace()
            map_score = 0.0
            for frame_t in range(inputs.shape[1]):
                map_x, embedding, x_Block1, x_Block2, x_Block3, x_input =  model(inputs[:,frame_t,:,:,:])
                score_norm = criterion_MSE(map_x, test_maps[:,frame_t,:,:])
                map_score += score_norm
            map_score = map_score/inputs.shape[1]

            map_score_list.append('{} {}\n'.format(map_score, spoof_label[0][0]))

        map_score_test_filename2 = args.log+'/'+ args.log+'_map_score_test2.txt'
        with open(map_score_test_filename2, 'w') as file:
            file.writelines(map_score_list)    

        #############################################################     
        #       performance measurement both val and test
        #############################################################     
        val_threshold, test_threshold, val_ACC, val_ACER, test_ACC, test_APCER, test_BPCER, test_ACER, test_ACER_test_threshold = performances(map_score_val_filename2, map_score_test_filename2)

        print('epoch:%d, Val:  val_threshold= %.4f, val_ACC= %.4f, val_ACER= %.4f' % (epoch, val_threshold, val_ACC, val_ACER))
        log_file.write('\n epoch:%d, Val:  val_threshold= %.4f, val_ACC= %.4f, val_ACER= %.4f \n' % (epoch + 1, val_threshold, val_ACC, val_ACER))

        print('epoch:%d, Test:  ACC= %.4f, APCER= %.4f, BPCER= %.4f, ACER= %.4f' % (epoch, test_ACC, test_APCER, test_BPCER, test_ACER))
        #print('epoch:%d, Test:  test_threshold= %.4f, test_ACER_test_threshold= %.4f\n' % (epoch + 1, test_threshold, test_ACER_test_threshold))
        log_file.write('epoch:%d, Test:  ACC= %.4f, APCER= %.4f, BPCER= %.4f, ACER= %.4f \n' % (epoch, test_ACC, test_APCER, test_BPCER, test_ACER))
        #log_file.write('epoch:%d, Test:  test_threshold= %.4f, test_ACER_test_threshold= %.4f \n\n' % (epoch + 1, test_threshold, test_ACER_test_threshold))
        log_file.flush()

log_file.close()
print("Finished!")

/opt/conda/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
/opt/conda/lib/python3.8/site-packages/torch/nn/functional.py:3609: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  warnings.warn(


epoch:1, Val:  val_threshold= 13317.5361, val_ACC= 0.9989, val_ACER= 0.0028
epoch:1, Test:  ACC= 1.0000, APCER= 0.0000, BPCER= 0.0000, ACER= 0.0000


/opt/conda/lib/python3.8/site-packages/torch/nn/functional.py:3609: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  warnings.warn(


epoch:51, Val:  val_threshold= 13223.8213, val_ACC= 0.9989, val_ACER= 0.0028
epoch:51, Test:  ACC= 1.0000, APCER= 0.0000, BPCER= 0.0000, ACER= 0.0000


/opt/conda/lib/python3.8/site-packages/torch/nn/functional.py:3609: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  warnings.warn(


...

epoch:401, Val:  val_threshold= 13220.4434, val_ACC= 0.9989, val_ACER= 0.0028
epoch:401, Test:  ACC= 1.0000, APCER= 0.0000, BPCER= 0.0000, ACER= 0.0000


/opt/conda/lib/python3.8/site-packages/torch/nn/functional.py:3609: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  warnings.warn(


epoch:451, Val:  val_threshold= 13216.6025, val_ACC= 0.9989, val_ACER= 0.0028
epoch:451, Test:  ACC= 1.0000, APCER= 0.0000, BPCER= 0.0000, ACER= 0.0000


/opt/conda/lib/python3.8/site-packages/torch/nn/functional.py:3609: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  warnings.warn(


epoch:501, Val:  val_threshold= 13219.8096, val_ACC= 0.9989, val_ACER= 0.0028
epoch:501, Test:  ACC= 1.0000, APCER= 0.0000, BPCER= 0.0000, ACER= 0.0000
Finished!
